# CHEM 60 - April 3rd, 2024 (The Metropolis Algorithm)

You thought about random numbers in the homework from last week and in class on Monday. Today, we're looking at a particularly important example of random numbers in computing (and chemistry).

The Metropolis algorithm (better known as Metropolis-Hastings these days) is a very well-known example of a Markov chain Monte Carlo (MCMC) method with wide-ranging applications in the computational sciences.

Look at all the fields that write papers about Metropolis Hastings:
![](https://kavassalis.space/s/MH_dimensionsAI_Apr24.png)

It also... comes from chemistry. No seriously. The Metropolis algorithm, this extremely general method that is used in things like pseudo-random number generators, numerical integration schemes, and finding minimal paths through complex surfaces, comes from a chemistry paper. Nicholas Metropolis and colleagues published a paper in the Journal of Chemical Physics in 1953 called "Equation of State Calculations by Fast Computing Machines" ("Fast Computing Machines" - adorable... They were talking about vacuum tube computers, fyi). The paper presented the first computational simulation of a liquid and did so by presenting a new Monte Carlo technique (you may have even learned about it in intro CS, but didn't give it its full name). This method was inspired by good old-fashioned physical chemistry - thinking about molecules moving around, and what determins how they move.

Save your in-class notebook copy in your personal Drive as usual.


#Imports

Here are the Python imports that we will need today. We don't need much to do this! The default formatting stuff is here too.

Run the below code block to get started.

In [ ]:
# Standard library imports
import math as m

# Third party imports
import matplotlib.animation as animation
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np
from scipy.stats import chisquare

# so the notebook shows an animation
rc('animation', html='html5')

# This part of the code block is telling matplotlib to make certain font sizes exra, extra large by default
params = {'legend.fontsize': 'xx-large',
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
# This line updates the default parameters of pyplot (to use our larger fonts)
plt.rcParams.update(params)

Not too much of this today. We'll make a bunch of animations to visualize things though.



---



# What is a Monte Carlo Method again?

Monte Carlo methods encompass a broad class of computational algorithms that use random sampling to obtain numerical results. These techniques are useful for solving complex tasks in pretty much every field that has numerical problems. The basic principle that all Monte Carlo methods share: you can approximate solutions to quantitative questions through the statistical analysis of random variables.

You probably saw a few examples of Monte Carlo methods in your intro CS course. Here is one below that will hopefully be familar.

![A slide from CS5 showing a Monte Carlo Method used for estimating pi](https://kavassalis.space/s/cs5_lec4_SP2022_MCMpi.png)
> *A slide from [CS5 in 2022](https://drive.google.com/file/d/1uKyrx67z9A3-YXvboo78uNKcpkZ83uNX/view) showing how you can use random pin drops to estimate the value of pi*. The semester most of you took CS5 has similar examples: https://www.cs.hmc.edu/twiki/bin/view/CS5Fall2023

The point of reminding you of this? Randomly dropping pins on a square with a circle drawn in it is 1) a valid way to estimate pi and 2) an example of a Monte Carlo method!



## Monte Carlo Example - Estimating Pi

If you haven't seen this before, or if you have, but want to see code to recreate the above simulation, read on! Given a unit square, the circle inside it will have a radius of 1/2. The area of the  circle is πr^2 (with r=1, this simplifies to π/4). Even more trivially, the area of the unit square is 1. By randomly placing points within the square and counting how many fall inside the quarter circle versus the total number of points, we can estimate π as 4 times the ratio of points inside the circle to the total points.

I have put a lot of comments on this code, to step through what each line is doing here.

In [ ]:
def MC_estimate_of_pi(frame):
    # The global keyword allows a variable to be used globally in your entire program.
    global total_points, inside_points

    # Generate two random float numbers between 0 (inclusive) and 1 (exclusive) using NumPy's random.rand() function.
    # Multiply these random points by 'square_size' and then subtract half of 'square_size' to shift the square's center to (0,0).
    x, y = np.random.rand(2) * square_size - square_size/2  # Shift to center at (0,0)

    # Calculate the Euclidean distance from the point (x, y) to the origin (0, 0).
    # It is accomplished by squaring the coordinates (x and y), summing them up, and taking the square root of the result.
    distance = np.sqrt(x**2 + y**2)  # Distance from point to origin

    # Check if the distance of the point from the origin is less or equal to half of the square size.
    # If true, it means the point is inside the circle, and increase the 'inside_points' counter.
    if distance <= square_size/2:
        inside_points += 1
          # Append the x and y coordinates to the data of points inside the circle for plot.
        points_inside.set_data(np.append(points_inside.get_xdata(), x),
                               np.append(points_inside.get_ydata(), y))
     # If the point is not inside the circle, it must be outside (but still inside the square).
    else:
        # Append the x and y coordinates to the data of points outside the circle for plot.
        points_outside.set_data(np.append(points_outside.get_xdata(), x),
                                np.append(points_outside.get_ydata(), y))

    # Increase the counter for the total number of generated points.
    total_points += 1
    # Calculate the estimation of pi by multiplying 4 by the ratio of points inside the circle to total points.
    estimated_pi = 4 * (inside_points / total_points)
    # Set the title of the plot with the estimated value of pi.
    # The title contains a string that describes the estimation of pi and the estimated value of pi with 5 decimal places precision.
    ax.set_title(f'Estimating Pi using Monte Carlo Simulation\nPi ≈ {estimated_pi:.5f}')

    # Return the updated data for points inside and outside the circle.
    return points_inside, points_outside

Let's run the code below. Since this is making an animation, it will be slow to run! The actual simulation here is really fast to run, but creating and displaying the animation is a slog. You can comment out the `ani` line at the very bottom of the below code block to see that for yourself (but you probably want to look at the animation, so, it'll be worth waiting).

Lots and lots of comments again (perhaps some of you have expressed excitement at making animations for your final project and wanted to see more code!)

In [ ]:
# Define the domain
square_size = 1 # The side length of the square is defined.

# Initialize plot
# A figure and a set of subplots (in this case, just one) are created using this function from matplotlib.pyplot module.
fig, ax = plt.subplots()
ax.axis('square') # This sets the aspect of the scale or the axis.
# Define the limits of x and y-axis between negative half of the square size and positive half.
ax.set_xlim(-square_size/2, square_size/2)
ax.set_ylim(-square_size/2, square_size/2)

# A circle is created with centre at origin and radius half the square size.
# This circle is not filled and has colour specified by colour code (a very good orange).
circle = plt.Circle((0, 0), square_size/2, color='#d95f02', fill=False)
ax.add_artist(circle) # This circle "artist" (matplotlib jargon for adding things like this to plots) is added to the plot using the add_artist method from the Axes class.

# Two empty scatter plots are prepared for points inside and outside the circle.
# They are each assigned their own colours (with hex), a marker shape, and no line (that's wat the empty string is doing '')
points_inside, = ax.plot([], [], '#7570b3', marker='o', linestyle='', label='Inside')
points_outside, = ax.plot([], [], '#1b9e77', marker='o', linestyle='', label='Outside')

# Two variables to hold the total number of points and the number of points that fall within the circle.
total_points = 0
inside_points = 0

# This now creates an animation by repeatedly calling a function (in this case, MC_estimate_of_pi).
# 'fig' is the figure object on which the animation will be drawn (we had to make that above).
# 'frames' is the number of frames (or iterations) through which the animation will be run. In this case, it is 100.
# 'blit' if True, will only re-draw the parts that have changed. This makes the animation more efficient.
# 'interval' controls the delay between frames in milliseconds.
# 'repeat' if False, means that the animation will not start again once it has finished.
ani = animation.FuncAnimation(fig, MC_estimate_of_pi, frames=range(100),
                              blit=True, interval=10,
                              repeat=False)

plt.close()
ani

## PRACTICE QUESTION

Change some arguments within FuncAnimation and see what they do! Can you make the pins drop more slowly? Can you make the animation drop more pins? Which arguments control those things?



---



In case you did not want to sit around to wait for the above to make a longer movie, I ran it for 2000 pin drops and have shared it [here](https://drive.google.com/file/d/1U96eDLv-PK64S_AKcxloSOjyUsMxhWcO/view?usp=drive_link).

# What is Metropolis-Hastings?

Let's work through this as a group. While the Metropolis paper is short, the language use is challenging. It also doesn't sound very similar to how you might learn about this algorithm in a CS class (but it's really the same). We'll use a [Google Doc](https://docs.google.com/document/d/17zF6hz8QKOtMpK8srcqxbitltNHUusOeewQspzKykO4/edit?usp=sharing). Get into pairs and sign up to read and summarize just one of the assigned sections.

Take some notes for yourself: What is the Metropolis algorithm supposed to do. How is this described in the de-chemistried version (Metropolis-Hastings) and how is it different than the classic, chemistry way of describing it.


---



**Your summary notes from our class discussion - your section notes can just be right in the Doc**

# Middle-Square Method

The 'approximating pi' Monte Carlo example takes advantage of numpy's [random](https://numpy.org/doc/stable/reference/random/generated/numpy.random.rand.html) numbers that you have used a lot lately. In the original Metropolis et al. paper, they obviously couldn't do this (this was the area of Fast Computing Machines™, after all). Instead, the authors used a method known as the "middle-square method" for generating pseudo-random numbers. The middle-square method, typically attributed to John von Neumann, involves taking a number (the "seed"), squaring it, and then taking the middle digits of the result as the next number in the series, which serves as both the next seed and the "random" number. This method is relatively simple but suffers from some major drawbacks, which you can test out below. I also genuinely recommend reading the [Wikipedia](https://en.wikipedia.org/wiki/Middle-square_method) page on the middle-square method too because it pulls no punches in calling it "highly flawed" and also suggests that perhaps it was initially invented in the 1200s by a Franciscan friar `¯\_(ツ)_/¯`.

Below is a small implementation of this in Python. Even though it is a tiny function, I have added lots of comments to explain what each part is doing there.

In [ ]:
def middle_square(seed, digits=4, steps=10):
    numbers = [] # Initialize an empty list to hold the random numbers that will be generated.
    for _ in range(steps): # Loop over the range of steps given.
        # The zfill method pads the string on the left with zeroes to ensure it has a length of digits*2 (so it has a middle).
        square = str(seed**2).zfill(digits*2) # Square the seed value and convert it to a string.
        # The starting and ending indices for the slice are calculated based on the length of the squared string and the number of digits.
        middle = int(square[(len(square)//2-digits//2):(len(square)//2+digits//2)]) # Extract the 'middle' digits from the squared string using slicing.
        numbers.append(middle) # Add the extracted middle number to the list of random numbers.
        seed = middle # The extracted middle number becomes the seed for the next iteration.
    return numbers  # Return the list of generated random numbers.

Let's test it out!

In [ ]:
seed = 1234  # initial seed
digits = 4  # number of digits to keep
steps = 10  # sequence length
random_numbers = middle_square(seed, digits, steps)
print("Pseudo-random sequence:", random_numbers)

## PRACTICE QUESTION

If you started with a seed of 1234, you should have got your first pseudo-random number as '5227'.  Double check that the math makes sense (ie. if you were to manually attempt to use the middle-squares method, you'd get the same next number). You can use a code block to square the numbers.


---





---



## PRACTICE QUESTION

Are these numbers "random enough"? Create a histogram and use the uniformity test to say if this method produces reasonably random numbers?


---



In [ ]:
# code here!



---



## PRACTICE QUESTION
Why does Wikipedia call this "deeply flawed" though? Increase the number of steps and see what happens. Are they still 'random enough'?


---





---

In case you really miss the [HMMM](https://www.cs.hmc.edu/~cs5grad/cs5/hmmm/documentation/documentation.html) from CS5/42, check out the code that Metropolis wrote to do this in the 1950s.

![Assembly for the middle-square algorithm as written for MANIAC](https://kavassalis.space/s/MANIAC-middle-square-program-listing.png)

This does not spark joy for me, but maybe it does for you! If so, you can read about the inner workings of the computer that was used for these very early chemical simulations. It was named... [MANIAC](https://drive.google.com/file/d/1S_Pqi5tbFIExufNAR-qBbm4s6vUa3eBv/view?usp=drive_link).

Moving on.

# Middle-Square Particle Simulation

Before we do a friendly version of Metropolis, let's try out the middle-square random numbers for a very simple simulation.

This function is being written to animate, so includes arguments to make that possible:

* `frame` parameter is required for matplotlib's FuncAnimation function, we do not actually use it in this function.
* `positions` is a list of tuples representing the current positions of particles.
* `square_size` is the side length of the square (part of the domain where particles move).
* `seed` is the initial number for the pseudorandom number generator (Middle Square method).
* `steps_per_frame` is the number of simulation steps performed in each frame of the animation.
* `digits` is the number of digits to be used in the Middle Square random number generation method (affects the total available numbers).


In [ ]:
# Update function utilizing middle_square for random numbers
def Middle_Square_Particle_Simulation(frame, positions, square_size, seed, steps_per_frame=1, digits=4):
    # For each simulation step in the current frame...
    for _ in range(steps_per_frame):
        # Generate pseudorandom numbers using Middle Square method.
        # The number of numbers generated is 2 times the number of particles.
        # Half these numbers will be used as steps in x direction, the other half - in y direction.
        rand_steps = middle_square(seed, digits=digits, steps=2*len(positions))
        # The seed for the next frame is set to the last generated number for continuity in random number generation.
        seed = rand_steps[-1]
         # Normalize steps in x and y direction (dividing by the maximum possible number in Middle Square method)
        # and shift them so they now range in [-1, 1). This transformation forms the dx steps in x direction.
        dxs = [(step / 10**digits) * 2 - 1 for step in rand_steps[:len(positions)]]  # Normalize and shift to [-1, 1)
        dys = [(step / 10**digits) * 2 - 1 for step in rand_steps[len(positions):]]  # Normalize and shift to [-1, 1)

        # For each particle and corresponding steps in x and y direction...
        for i, (dx, dy) in enumerate(zip(dxs, dys)):
            # Retrieve the current position of the i-th particle
            x, y = positions[i]
            # Update its position by adding the corresponding step.
            # Modulo operation (%) ensures that particles remain inside the square - if a particle moves past an edge of the square, it appears on the opposite side (this is known as a periodic boundary condition).
            new_x, new_y = (x + dx) % square_size, (y + dy) % square_size
            # Save the new position of the i-th particle.
            positions[i] = (new_x, new_y)

    # Update the positions of points in the scatter plot.
    scat.set_offsets(positions)
    # In matplotlib animations a tuple should be returned. The trailing comma is typically used for one element tuples.
    # The scat object must be returned as it's updated in-place. 'scat' is the variable which holds the reference for our scatter plot.
    return scat,

Let's see what this does

In [ ]:
# A new figure and a subplot are created using the matplotlib's subplots function.
fig, ax = plt.subplots()
# Variable 'square_size' is the side length of the square in which particles can move.
square_size = 10
# The initial seed for the pseudorandom number generator (Middle Square method).
rng_seed = 1234

# The initial positions of 50 particles are generated.
# np.random.rand(50, 2) creates an array of size (50, 2) with random float numbers in the range [0, 1).
# By multiplying it by 'square_size', the range of random numbers become [0, 'square_size').
positions = np.random.rand(50, 2) * square_size  # Initial positions for 50 particles

# For each particle, assign a colour from the jet colourmap which ranges from blue to red.
colours = [cm.jet(i/len(positions)) for i in range(len(positions))]

# 'scatter' function creates a scatter plot.
# With 'positions[:, 0]' we get all x coordinates,
# and with 'positions[:, 1]' we get all the y coordinates of the particles.
# 'c=colours' assigns the previously defined colours to the particles.
scat = ax.scatter(positions[:, 0], positions[:, 1], c = colours)

# Set the limits of x-axis and y-axis to [0, square_size).
ax.set_xlim(0, square_size)
ax.set_ylim(0, square_size)

# Create an animation using FuncAnimation function from matplotlib.
ani = animation.FuncAnimation(fig, Middle_Square_Particle_Simulation, fargs=(positions, square_size, rng_seed),
                              frames=200, interval=100, blit=True)

plt.close()
ani

Note on the FuncAnimation arguments:

* In `fargs` parameter, we provide a tuple of arguments that will be passed to Middle_Square_Particle_Simulation function.
* `frames=200` means that the simulation will consist of 200 frames.
* `interval=100` means that the delay between frames will be 100 milliseconds.
* `blit=True` means that only parts of the figure that have changed will be re-drawn, which can improve performance of an animation.

Okay. So, that looks kind of fun. But we also said that our pseudo-random number generator was "deeply flawed". Let's just recreate that simulation using a more modern pseudo-random number generator.

In [ ]:
def NumpyRand_Particle_Simulation(frame, positions, square_size, seed, steps_per_frame=1):
    for _ in range(steps_per_frame):
        np.random.seed(seed)
        # Generate random steps for each particle using numpy's random.rand()
        rand_steps = np.random.rand(2 * len(positions))

        # Convert rand_steps to a list to use list slicing
        rand_steps = rand_steps.tolist()

        dxs = [(step * 2) - 1 for step in rand_steps[:len(positions)]]  # Normalize and shift to [-1, 1)
        dys = [(step * 2) - 1 for step in rand_steps[len(positions):]]  # Normalize and shift to [-1, 1)

        for i, (dx, dy) in enumerate(zip(dxs, dys)):
            x, y = positions[i]
            new_x, new_y = (x + dx) % square_size, (y + dy) % square_size
            positions[i] = (new_x, new_y)

    # Update scatter plot
    scat.set_offsets(positions)
    return scat,

## PRACTICE QUESTION

What is different in the code of `NumpyRand_Particle_Simulation` function from the `Middle_Square_Particle_Simulation` function?


---



**notes**



---



Let's look at the simulation.

In [ ]:
# Initializing the plot
fig, ax = plt.subplots()
square_size = 10
rng_seed = 1234  # Initial seed for the middle_square method
positions = np.random.rand(50, 2) * square_size  # Initial positions for 50 particles

scat = ax.scatter(positions[:, 0], positions[:, 1], c = colours)
ax.set_xlim(0, square_size)
ax.set_ylim(0, square_size)

ani = animation.FuncAnimation(fig, NumpyRand_Particle_Simulation, fargs=(positions, square_size, rng_seed),
                              frames=200, interval=100, blit=True)

plt.close()
ani

## PRACTICE QUESTION:
Qualitatively, these look quite similar! Do you notice any differences?



---



**notes**



---



## PRACTICE QUESTION

What is `np.random.seed(seed)` doing here? And why might I always ask students to set and state the PRNG seeds in research applications?



---



**notes**



---



## PRACTICE QUESTION
Comment out the `np.random.seed(seed)` line. Describe what is different between the with and without 'set random seed line' animations? What physical scenarios could the two different simulations describe?



---



**Notes**



---



## PRACTICE QUESTION

Try changing where you set the numpy random seed (ie. not within the for-loop). Use the same seed as your neighbour and figure out where you would need to set the random seed so you both could get the exact same simulation.


---



In [ ]:
# maybe code



---



# Metropolis

As we hopefully came to conclude above, the simplified version of Metrpolis-Hastings is, say we want to move molecules around (or predict the next number in a sequence, ie. anything we want to iterate, really). We can describe that process in the language of PCHEM.
* $x$ is the state of the system
* $f(x)$ is the energy of the system
* and $T$ is the temperature and $k$ the Boltzman constant (so $kT$ is essentially temperature in the units of energy)

Now the algorithm should address how the system might evolve ($x'$ is a potential next state for the system):

1.   $x' = x + $ a random change
2.   If $\text{exp}\frac{f(x)-f(x')}{kT} > $ A random number, then $x = x'$ (ie. if the energy of the system in the state $x'$ is lower than the energy of the system in state $x$, move to the new state.

And repeat.

You have to have some way of calculating energy. For many applications of this algorithm, we're not talking about real energy in the way you do for molecular populations - we are just talking about some quantity we want to minimize (a cost function, a loss function <- you've seen these before in this course).

I will show you a simple version of the Metropolis algorithm for essentially just moving non-interacting particles around. In the paper, the molecules being rigid bodies and on a lattice (to try and simulate liquid properties and molecules actually having size) is important and not included below (but the basic logic is the same for all implementations of Metropolis-Hastings).

Let's step through it a bit more carefully:

1. **Calculate the Initial Energy**: Initially calculate the energy of the system before any moves are made.

Why don't we define the energy of the system as just being a function of the distances between all the particles (we're ignoring some things here! But this is a simple demo)

In [ ]:
def calculate_energy(positions):
    # Example: Sum of distances between particles
    energy = sum(np.sqrt((x1-x2)**2 + (y1-y2)**2) for (x1, y1), (x2, y2) in zip(positions[:-1], positions[1:]))
    return energy

2. **Generate Random Steps**: Utilize the `middle_square` method to generate random steps for particle movement, as you already do.
3. **Attempt Particle Moves**: For each particle, compute its new position based on the generated steps.
4. **Calculate New Energy**: Calculate the energy of the system if the particle were to move to its new position.
5. **Apply the Metropolis Condition**: Compare the new energy with the initial energy, and decide whether to move the particle based on the Metropolis algorithm criteria.
6. **Update Positions**: If a move is accepted, update the particle's position.

We will use the numpy random numbers for this one...

In [ ]:
def Metropolis_Particle_Simulation(frame, positions, square_size, seed, kT=1, steps_per_frame=1, digits=4):
    # Calculate the initial energy state
    initial_energy = calculate_energy(positions)

    # For each simulation step in the current frame...
    for _ in range(steps_per_frame):
        # Generate pseudorandom steps for each particle.
        # Total number generated is two times number of particles.
        # Half these numbers will be used as steps in x direction, the other half - in y direction.
        rand_steps = 10**digits * np.random.rand(2*len(positions))

        # These steps are normalized (dividing by the maximum possible number
        # considering the number of digits, then scaled up by 2, finally subtracting 1) to have a random range between [-1, 1).
        # This forms the dx (change in x-direction).
        dxs = [(step / (10**digits / 2)) - 1 for step in rand_steps[:len(positions)]]

        # Similarly normalized steps for dy (change in y-direction).
        dys = [(step / (10**digits / 2)) - 1 for step in rand_steps[len(positions):]]

        # For each particle and corresponding steps in x and y direction...
        for i, (dx, dy) in enumerate(zip(dxs, dys)):
            # Update the position of the particle by adding the corresponding step (dx, dy) and wrapping positions according to periodic boundary conditions.
            x, y = positions[i]
            new_x, new_y = (x + dx) % square_size, (y + dy) % square_size

            # Copy the current state of the system to evaluate potential new state
            new_positions = positions.copy()
            new_positions[i] = (new_x, new_y)

            # Calculate new energy of the system for the potential move
            new_energy = calculate_energy(new_positions)

            # Metropolis condition: if energy decreases, move is accepted. If energy increases, it is accepted with a probability exp(-ΔE/kT)
            # where ΔE is the change in energy, kT is the product of Boltzmann constant and absolute temperature
            if new_energy <= initial_energy or np.random.rand() < np.exp(-(new_energy - initial_energy) / kT):
                # move the i-th particle to the new position
                positions[i] = (new_x, new_y)

                # Update energy to the new_energy for the next iteration
                initial_energy = new_energy

    # Update the positions of points in scatter plot.
    scat.set_offsets(positions)

    # Return the scatter plot. This pattern (returning a tuple) is required by matplotlib animations.
    return scat,

And let's test it out.

In [ ]:
# Define constants
kT = 1  # Define your system's "temperature" (our units are made up, given how we defined energy)
square_size = 10  # Define size of the system

positions = np.random.rand(50, 2) * square_size  # Initial positions for 50 particles

# Initialize colours
colours = [cm.jet(i/len(positions)) for i in range(len(positions))]

figure = plt.figure()
scat = plt.scatter(*zip(*positions), c=colours, animated=True)

# initial seed
seed = 1234
# Run animation
ani = animation.FuncAnimation(figure, Metropolis_Particle_Simulation, frames=200, fargs=( positions, square_size, seed), repeat=False)
plt.close()
ani



---



This looks so different!

What if - instead of starting with randomly distributed particles - we started with them on a lattice:

In [ ]:
# Initialize particles in a regular lattice
N = 7  # Define the number of particles in each dimension
spacing = square_size / N  # Calculate the spacing between particles
# We are using a list comprehension that generates a tuple (x, y) for each point in the grid.
positions = np.array([(i*spacing, j*spacing) for i in range(N) for j in range(N)])

# Initialize colours
colours = [cm.jet(i/len(positions)) for i in range(len(positions))]

figure = plt.figure()
scat = plt.scatter(*zip(*positions), c=colours, animated=True)
plt.title("Initial Conditions on a Lattice")
plt.show()

What happens now?

In [ ]:
# initial seed
seed = 1234
# Run animation
ani = animation.FuncAnimation(figure, Metropolis_Particle_Simulation, frames=200, fargs=( positions, square_size, seed), repeat=False)
plt.close()
ani

## PRACTICE QUESTION:

What kinds of questions could this basic framework help us answer? What kinds of complexity could you add to this type of simulation to tell us information about real systems?



---



**notes**



---



# Submit your notebook

It's time to download your notebook and submit it on Canvas. Go to the File menu and click **Download** -> **Download .ipynb**

Then, go to **Canvas** and **submit your assignment** on the assignment page. Once it is submitted, swing over to the homework now and start working through the paper.